In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt

import numpy as np
import pandas as pd; pd.options.mode.chained_assignment = None

import APIs.Bloomberg as ba
import Weather.W_USA as wu

import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import plotly.express as px

In [7]:
tl=['danilele','moline','damon','semen']

In [9]:
tl[-2:-1]

['damon']

In [6]:
test=' and '.join(tl)
test

'danilele and moline and damon'